# Around the world in X number of days using Traveling Salesman Problem (TSP)

In this tutorial, we will learn how to exactly and heuristically solve TSP. 

In [1]:
# Copyright 2020, Gurobi Optimization, LLC
import csv, time, math, random  
from gurobipy import * 
import networkx as nx # For drawing tours and subtours
from haversine import haversine, Unit # For computing the haversine distance

### Reading data

In [2]:
capitals = {} # Dictionary of capitals with values as coordinates
with open('coord_ind_cap.csv', newline='') as csvfile:
    reader = csv.reader(csvfile) # reader
    next(reader) # skip header
    for row in reader:
        capitals[row[1], row[0]] = (float(row[2]), float(row[3]))

dist_matrix = {}
for c1 in capitals:
    for c2 in capitals:
        dist_matrix[c1, c2] = haversine(capitals[c1], capitals[c2], unit=Unit.KILOMETERS) # computes the haversine distance between two coordinates

neighbors = {i:[] for i in capitals} # Adjacency dictionary

for i, j in dist_matrix:
    if j not in neighbors[i]:
        neighbors[i].append(j)
    if i not in neighbors[j]:
        neighbors[j].append(i)

In [3]:
# Drawing on the map
# Drawing the tour/subtour
import folium # Library for creating the maps
def draw_tour(edges):     
    # The averate coordinate will be used for setting up the initial map
    average_lat = sum(lat for lat, lon in capitals.values()) / len(capitals)
    average_lon = sum(lon for lat, lon in capitals.values()) / len(capitals)
    
    # Intialize the map with location of the center of the map and zoom level
    mymap = folium.Map(location=[average_lat, average_lon], zoom_start=4)
    
    # We put a small circle at each capital.   
    for c in capitals:
        folium.Circle([capitals[c][0], capitals[c][1]],radius=2000, color='red', fill=True, fill_color='red', 
                      fill_opacity=0.7,popup=c).add_to(mymap)

    # We put edges between coordinates
    for i, j in edges:
        coord1 = (capitals[i][0], capitals[i][1])
        coord2 = (capitals[j][0], capitals[j][1])
        folium.PolyLine([coord1, coord2], color="blue", weight=2.5, opacity=1).add_to(mymap)
        
    return mymap

### MTZ model


In [4]:
m = Model() # Initialize the model

# Define decision variables
x = {(i, j): m.addVar(vtype = GRB.BINARY) for i in capitals for j in capitals} # Whether an edge is part of tour or not
u = {i: m.addVar(vtype=GRB.INTEGER) for i in capitals} # Order in which each capital is visited

# Add constraints
for i in capitals:
    m.addConstr(sum([x[i, j] for j in capitals if i != j]) == 1) 
    m.addConstr(sum([x[j, i] for j in capitals if i != j]) == 1)

# Without loss of generality, let's assume the first city is the first capital in the list
first = list(capitals.keys())[0]

m.addConstr(u[first] == 1) # The order for first city is 1


for i in capitals:
    if i != first:
        m.addConstr(u[i] <= len(capitals)) # Order should be less than the number of cities
        m.addConstr(u[i] >= 2)

for i, j in dist_matrix:
    if i != first and j != first:
        m.addConstr(u[i] - u[j] + 1 <= (len(capitals) - 1) * (1 - x[i, j])) # Subtour elimination


obj = sum([dist_matrix[k] * x[k] for k in dist_matrix]) # Total cost of tour
m.setObjective(obj, sense=GRB.MINIMIZE)
m.optimize()
print(m.status)
edges  = [k for k in x if x[k].x > 0.4] # Edges in the optimal tour

Set parameter Username
Academic license - for non-commercial use only - expires 2025-10-12
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: 13th Gen Intel(R) Core(TM) i9-13900, instruction set [SSE2|AVX|AVX2]
Thread count: 24 physical cores, 32 logical processors, using up to 32 threads

Optimize a model with 1224 rows, 1190 columns and 5512 nonzeros
Model fingerprint: 0xf0524df0
Variable types: 0 continuous, 1190 integer (1156 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+01]
  Objective range  [6e+01, 3e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+01]
Presolve removed 100 rows and 35 columns
Presolve time: 0.00s
Presolved: 1124 rows, 1155 columns, 5412 nonzeros
Variable types: 0 continuous, 1155 integer (1122 binary)
Found heuristic solution: objective 48270.923208

Root relaxation: objective 1.137562e+04, 127 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work

In [87]:
draw_tour(edges) 

### Formulation with alternative subtour constraints


In [8]:
m = Model()
x = {(i, j): m.addVar(vtype = GRB.BINARY, name = f'x_{i},{j}') for i in capitals for j in capitals}
q = {(i, j): m.addVar(vtype = GRB.CONTINUOUS, name=f'q_{i},{j}') for i in capitals for j in capitals}

for i in capitals:
    m.addConstr(sum([x[i, j] for j in capitals if j != i]) == 1)
    m.addConstr(sum([x[j, i] for j in capitals if j != i]) == 1)

first = list(capitals.keys())[0]

m.addConstr(sum([q[first, j] for j in capitals]) == len(capitals), name = 'len')
m.addConstr(sum([q[j, first] for j in capitals]) == 1, name = 'first')

for i in capitals:
    if i != first:
        m.addConstr(sum([q[j, i] for j in capitals if j != i]) - sum([q[i, j] for j in capitals if j != i]) == 1, name = f'cons_{i}')

for i in capitals:
    for j in capitals:
        if i != j:
            m.addConstr(q[i,j] <= (len(capitals) + 2) * x[i, j])

obj = sum([dist_matrix[k] * x[k] for k in dist_matrix])
m.setObjective(obj, sense=GRB.MINIMIZE)
m.optimize()

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: 13th Gen Intel(R) Core(TM) i9-13900, instruction set [SSE2|AVX|AVX2]
Thread count: 24 physical cores, 32 logical processors, using up to 32 threads

Optimize a model with 1225 rows, 2312 columns and 6734 nonzeros
Model fingerprint: 0x3b11d716
Variable types: 1156 continuous, 1156 integer (1156 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+01]
  Objective range  [6e+01, 3e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+01]
Presolve removed 0 rows and 67 columns
Presolve time: 0.01s
Presolved: 1225 rows, 2245 columns, 6734 nonzeros
Variable types: 1123 continuous, 1122 integer (1122 binary)
Found heuristic solution: objective 62866.638909
Found heuristic solution: objective 38930.395746

Root relaxation: objective 1.168296e+04, 1937 iterations, 0.04 seconds (0.04 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | 

In [9]:
edges  = [k for k in x if x[k].x > 0.4]
draw_tour(edges)

## DFJ

In [46]:
from itertools import combinations # for obtaining subsets

m = Model()
x = {(i, j): m.addVar(vtype = GRB.BINARY) for i in capitals for j in capitals if i != j}

for i in capitals:
    m.addConstr(sum([x[i, j] for j in capitals if i != j]) == 1)
    m.addConstr(sum([x[j, i] for j in capitals if i != j]) == 1)


obj = sum([dist_matrix[k] * x[k] for k in dist_matrix if k[0] != k[1]])
m.setObjective(obj, sense=GRB.MINIMIZE)
m.Params.OutputFlag = 0

In [47]:
m.optimize()

In [48]:
edges  = [k for k in x if x[k].x > 0.4]
draw_tour(edges)

In [49]:
def find_subtours(edges):
    subtours = []
    while edges:
        (i, j) = edges.pop(0)
        visited = [i, j]
        stop = False
        while not stop:
            neighbors = [k for k in edges if k[0] == j]
            if neighbors[0][1] in visited:
                visited.append(neighbors[0][1])
                edges.remove(neighbors[0])
                subtours.append(visited)
                stop = True
            else:
                visited.append(neighbors[0][1])
                (i, j) = neighbors[0]
                edges.remove(neighbors[0])
    return subtours
iterations = 0
        

### We repeatedly add subtour elimination constraints until we get a tour

In [71]:
subtours = find_subtours(edges)

for s in subtours:
    tmp = 0; edge_count = 0
    for n in range(len(s)):
        if n != 0:
            tmp += x[s[n-1], s[n]]
            edge_count += 1
    m.addConstr(tmp <= edge_count - 1)

m.update()
m.optimize()
edges  = [k for k in x if x[k].x > 0.4]
draw_tour(edges)
subtours = find_subtours(edges)
iterations +=1 
print(iterations)


edges  = [k for k in x if x[k].x > 0.4]
draw_tour(edges)

22


In [72]:
print('TSP solved in %d iterations'%iterations)
print('cost of the tour is: %f'%m.objVal)

TSP solved in 22 iterations
cost of the tour is: 12695.760679


#### To sutomate this process, we route a subroutine "subtourelim" and pass it in a callback function. This is how you implement a version of branch-and-cut

In [74]:
# Callback - use lazy constraints to eliminate sub-tours
def subtourelim(model, where):

    if where == GRB.Callback.MIPSOL:
        vals = model.cbGetSolution(model._x) 

        
        
        edges  = [k for k in m._x if vals[k] > 0.4]
        subtours = find_subtours(edges)
        print('It: %d and # subtours: %d'%(m._iterations, len(subtours)))
        if len(subtours) > 1:
            for s in subtours:
                tmp = 0; edge_count = 0
                for n in range(len(s)):
                    if n != 0:
                        tmp += model._x[s[n-1], s[n]]
                        edge_count += 1
            
                model.cbLazy(tmp <= edge_count - 1)
            
            
        model._iterations += 1
        
        

In [75]:
m = Model()
m._x = {(i, j): m.addVar(vtype = GRB.BINARY) for i in capitals for j in capitals if i != j}
m._iterations = 0
for i in capitals:
    m.addConstr(sum([m._x[i, j] for j in capitals if i != j]) == 1)
    m.addConstr(sum([m._x[j, i] for j in capitals if i != j]) == 1)


obj = sum([dist_matrix[k] * m._x[k] for k in dist_matrix if k[0] != k[1]])
m.setObjective(obj, sense=GRB.MINIMIZE)
m.Params.lazyConstraints = 1
m.optimize(subtourelim)

Set parameter LazyConstraints to value 1
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: 13th Gen Intel(R) Core(TM) i9-13900, instruction set [SSE2|AVX|AVX2]
Thread count: 24 physical cores, 32 logical processors, using up to 32 threads

Optimize a model with 68 rows, 1122 columns and 2244 nonzeros
Model fingerprint: 0xa14f37ca
Variable types: 0 continuous, 1122 integer (1122 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e+01, 3e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
It: 0 and # subtours: 2
Presolve time: 0.00s
Presolved: 68 rows, 1122 columns, 2244 nonzeros
Variable types: 0 continuous, 1122 integer (1122 binary)
It: 1 and # subtours: 16
It: 2 and # subtours: 17

Root relaxation: objective 1.227256e+04, 106 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/No

In [76]:
edges  = [k for k in m._x if m._x[k].x > 0.4]
draw_tour(edges)

In [77]:
print('Cost of optimal tour:%f'%m.objVal)

Cost of optimal tour:12695.760679


## Nearest neighbor heuristic

In [78]:
def nearest_neighbor():
    best_tour = []; best_tour_cost = float("inf")
    for i in capitals:
        unvisited = [k for k in capitals if k != i]
        start = i
        tmp_tour = []; tmp_cost = 0
        while len(unvisited) > 0:   
            dist = {k: dist_matrix[k] for k in dist_matrix if k[0] == start and k[1] in unvisited}
            link = min(dist, key=dist.get)
            tmp_tour.append(link)
            tmp_cost += dist[link]
            unvisited.remove(link[1])
            start = link[1]
        tmp_tour.append((link[1], i))
        tmp_cost += dist_matrix[link[1], i]
        if best_tour_cost > tmp_cost:
            best_tour_cost = tmp_cost
            best_tour = tmp_tour
    return best_tour, best_tour_cost

In [79]:
best_tour, best_tour_cost = nearest_neighbor()
print(best_tour_cost)
draw_tour(best_tour)

13791.33048043776


## Sweep heuristic

In [80]:
def sweep():
    xcoord = [capitals[k][0] for k in capitals]
    ycoord = [capitals[k][1] for k in capitals]
    centres = []
    
    centres = [(sum(xcoord)/len(capitals) , sum(ycoord)/len(capitals)), (0.5*(max(xcoord) + min(xcoord)), 0.5*(max(ycoord) + min(ycoord)))]
    
    best_tour = []; best_tour_cost = float("inf")
    for xc, yc in centres:
        angle = {} # around the center
        for i in capitals:
            angle[i] = math.degrees(math.atan2(capitals[i][1] - yc, capitals[i][0] - xc))
        angle  = [k for k, v in sorted(angle.items(), key=lambda item: item[1])]
        tour = []; tour_cost = 0
        for k in range(len(angle)):
            if k != 0:
                tour.append((angle[k-1], angle[k]))
                tour_cost += dist_matrix[angle[k-1], angle[k]]
            else:
                start = angle[k]
        tour.append((angle[k], start))
        tour_cost += dist_matrix[(angle[k], start)]
        if tour_cost < best_tour_cost:
            best_tour_cost = tour_cost 
            best_tour = tour
    return best_tour, best_tour_cost
            
best_tour, best_tour_cost = sweep()
print(best_tour_cost)
draw_tour(best_tour)

16944.46175833539


## Savings algorithm

In [81]:
import itertools
start = max(capitals.items(), key=lambda item: (item[1][0], item[1][1]))[0]
unexplored = [k for k in capitals if k != start]
def savings():
    savings = {}
    for k in list(itertools.combinations(capitals, 2)):
        savings[k] = dist_matrix[k[0], start] + dist_matrix[start, k[1]] - dist_matrix[k]
    
    p, q = max(savings, key=savings.get)
    tour = [(p, q)]
    unexplored.remove(p)
    unexplored.remove(q)
    
    while unexplored:
        tmpDict1 = {h: (dist_matrix[p, start] + dist_matrix[start, h] - dist_matrix[p, h], dist_matrix[q, start] + dist_matrix[start, h] - dist_matrix[q, h]) for h in unexplored}
        
        tmpDict2= {}
        for k in tmpDict1:
            if tmpDict1[k][0] > tmpDict1[k][0]:
                tmpDict2[k, p] = tmpDict1[k][0]
            else:
                tmpDict2[k, q] = tmpDict1[k][1]
                
        a, b = max(tmpDict2, key=tmpDict2.get)
        
        if b == p:
            tour.insert(0, (a, p))
        else:
            tour.append((q, a))
        
        p = tour[0][0]
        q = tour[-1][1]
        unexplored.remove(a)
    
    
    
    tour.insert(0, (start, tour[0][0]))
    tour.append((tour[-1][1], start))

    return tour, sum([dist_matrix[k] for k in tour])

best_tour, best_tour_cost = savings()
print(best_tour_cost)
draw_tour(best_tour)


15118.210264739335


## Cheapest insertion

In [82]:
def cheapest_insertion():
    dist = {k:dist_matrix[k] for k in dist_matrix if k[0] != k[1]}
    p, q = min(dist, key=dist.get)
    tour = [(p, q)]
    print(p, q)
    unexplored = [k for k in capitals if k != p and k != q]
    while unexplored:
        point_cost = {}
        for k in unexplored:
            tmpCost = float("inf"); i = 'NA'; j = 'NA'
            for (a, b) in tour:
                if dist_matrix[a, k] + dist_matrix[k, b]- dist_matrix[a, b] <  tmpCost:
                    tmpCost = dist_matrix[a, k] + dist_matrix[k, b]- dist_matrix[a, b]
                    i = a; j = b
            point_cost[k, (i, j)] = tmpCost
        k, (i,j) = min(point_cost, key=point_cost.get)
        tour.remove((i,j))
        tour.append((i, k))
        tour.append((k,j))
        unexplored.remove(k)
    tour.append((p, q))
    return tour, sum([dist_matrix[k] for k in tour])

best_tour, best_tour_cost = cheapest_insertion()
print(best_tour_cost)
draw_tour(best_tour)

('Shimla', 'Himachal Pradesh') ('Chandigarh', 'Punjab')
13584.739353336783


## Priciest insertion

In [83]:
def priciest_insertion():
    dist = {k:dist_matrix[k] for k in dist_matrix if k[0] != k[1]}
    p, q = min(dist, key=dist.get)
    tour = [(p, q)]
    print(p, q)
    unexplored = [k for k in capitals if k != p and k != q]
    while unexplored:
        point_cost = {}
        for k in unexplored:
            tmpCost = float("inf"); i = 'NA'; j = 'NA'
            for (a, b) in tour:
                if dist_matrix[a, k] + dist_matrix[k, b]- dist_matrix[a, b] <  tmpCost:
                    tmpCost = dist_matrix[a, k] + dist_matrix[k, b]- dist_matrix[a, b]
                    i = a; j = b
            point_cost[k, (i, j)] = tmpCost
        k, (i,j) = max(point_cost, key=point_cost.get)
        tour.remove((i,j))
        tour.append((i, k))
        tour.append((k,j))
        unexplored.remove(k)
    tour.append((p, q))
    return tour, sum([dist_matrix[k] for k in tour])

best_tour, best_tour_cost = priciest_insertion()
print(best_tour_cost)
draw_tour(best_tour)

('Shimla', 'Himachal Pradesh') ('Chandigarh', 'Punjab')
13562.587061454724


## ## Nearest insertion

In [84]:
def nearest_insertion():
    dist = {k:dist_matrix[k] for k in dist_matrix if k[0] != k[1]}
    p, q = min(dist, key=dist.get)
    tour = [(p, q)]
    print(p, q)
    unexplored = [k for k in capitals if k != p and k != q]
    explored = [p, q]

    while unexplored:
        point_cost = {(a, b): dist_matrix[a, b] for a in unexplored for b in explored}
        k, j = min(point_cost, key=point_cost.get)
        
        point_cost = {}
       
        tmpCost = float("inf"); i = 'NA'; j = 'NA'
        for (a, b) in tour:
            if dist_matrix[a, k] + dist_matrix[k, b]- dist_matrix[a, b] <  tmpCost:
                tmpCost = dist_matrix[a, k] + dist_matrix[k, b]- dist_matrix[a, b]
                i = a; j = b
        tour.remove((i,j))
        tour.append((i, k))
        tour.append((k,j))
        unexplored.remove(k)
    tour.append((p, q))
    return tour, sum([dist_matrix[k] for k in tour])

best_tour, best_tour_cost = nearest_insertion()
print(best_tour_cost)
draw_tour(best_tour)

('Shimla', 'Himachal Pradesh') ('Chandigarh', 'Punjab')
13638.521699122834


## Farthest insertion

In [85]:
def farthest_insertion():
    dist = {k:dist_matrix[k] for k in dist_matrix if k[0] != k[1]}
    p, q = min(dist, key=dist.get)
    tour = [(p, q)]
    print(p, q)
    unexplored = [k for k in capitals if k != p and k != q]
    explored = [p, q]

    while unexplored:
        tmp2 = -float("inf"); k = 'NA'
        for a in unexplored:
            tmp1 = float("inf"); j = 'NA'
            for b in explored:
                if tmp1 > dist_matrix[a, b]:
                    tmp1 = dist_matrix[a, b]; j = b
            if tmp2 < tmp1:
                tmp2 = tmp1; k = a
                
        
       
        tmpCost = float("inf"); i = 'NA'; j = 'NA'
        for (a, b) in tour:
            if dist_matrix[a, k] + dist_matrix[k, b]- dist_matrix[a, b] <  tmpCost:
                tmpCost = dist_matrix[a, k] + dist_matrix[k, b]- dist_matrix[a, b]
                i = a; j = b
        tour.remove((i,j))
        tour.append((i, k))
        tour.append((k,j))
        unexplored.remove(k)
    tour.append((p, q))
    return tour, sum([dist_matrix[k] for k in tour])

best_tour, best_tour_cost = farthest_insertion()
print(best_tour_cost)
draw_tour(best_tour)

('Shimla', 'Himachal Pradesh') ('Chandigarh', 'Punjab')
13548.288647195177
